# User scraping 

- 강남역 맛집 리뷰 수가 많은 유저의 myplace를 크롤링 (목표 user 1000명)

- user_profile_df : User의 아이디, 리뷰수, 팔로워, url을 저장함
- user_df : '아이디', '리뷰', '팔로워', '매장명', '카테고리', '주소', '리뷰 내용', '세부정보', '태그', '방문일자', '재방문횟수' 를 저장함


### 개요 

1. 매장 정보 크롤링 결과에서 업태구분 별로 리뷰가 많은 식당 n개를 골라 가져온다
2. 리뷰가 많은 식당 n개에서 각각 3명씩 ( 추천순 or 최신순 일지는 아직 미정) 

In [130]:
# 웹 드라이버 설정
from selenium import webdriver  
from webdriver_manager.chrome import ChromeDriverManager 
from selenium.webdriver.common.action_chains import ActionChains

# 대기 관련 라이브러리
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC 
from selenium.webdriver.common.by import By

# 예외 처리 관련 라이브러리
from selenium.common.exceptions import TimeoutException, NoSuchElementException  

# 웹 요소 찾기 관련 라이브러리
from selenium.webdriver.common.by import By  

from selenium.webdriver.support.ui import Select  
from selenium.webdriver.common.keys import Keys  

# 그 외 
import time 
import warnings
warnings.filterwarnings('ignore')
from bs4 import BeautifulSoup 
import numpy as np  
import pandas as pd 
import re  
from tqdm import tqdm  # 반복문 진행 상황 시각화 모듈
import os
from datetime import datetime

import pandas as pd
from selenium.webdriver.common.by import By

### 함수 

In [1]:
# 정규 표현식으로 이모티콘, 특수문자, 아스키코드 제거
def remove_special_characters(text):
    pattern = r'[^\w\s]|_'
    result = re.sub(pattern, '', text)
    return result

In [131]:
# User의 아이디, 리뷰수, 팔로워, url을 가져오는 함수 

def user_profile(url) :
    # 주소 이동

    global driver
    driver.get(url)
    time.sleep(1)

    soup = BeautifulSoup(driver.page_source, 'html.parser')
    user_element = {}

    # User ID 찾기
    user_id_element = soup.find('button', class_='wTaI4v _2kK3N- _2we3hB')
    user_element['아이디'] = user_id_element.text if user_id_element else None

    # User의 인기도 : 리뷰수, 팔로워 찾기 
    user_popularity = soup.find_all('button', class_='wTaI4v _15qVKh')
    
    for element in user_popularity:
        em_tag = element.find('em')
        if em_tag:
            key = element.text.replace(em_tag.text, '').strip()
            value = int(re.sub('[^0-9]', '', em_tag.text))  # 쉼표 제거 후 변환
            user_element[key] = value
    
    # User URL
    user_element['url'] = url

    print(user_element)

    return user_element

def change_date_format(날짜) :
    #  부분 추출 (년월일까지)
    date_part = ""
    for i in 날짜 :
        date_part += i
        date_part += ','
    date_part = date_part.split('\n')[1]

    # 일자, 요일, 방문일수 분류하기 
    date_part = date_part.split(',')

    # '21.1.3.'에서 숫자 부분 추출
    revisit_match = re.search(r'(\d+\.\d+\.\d+)\.', date_part[1])
    if revisit_match:
        revisit_str = revisit_match.group(1)
        # 방문일수 정수로 변환
        revisit = int(revisit_str.replace('.', ''))
    else:
        revisit = 0  # 또는 다른 기본값으로 설정

    # 일자 타입 변경하기 
    date = date_part[0]
    weekday = date[-3:]
    match = re.findall(r'(\d+)년 (\d+)월 (\d+)일', date) # 타입 바꾸기 
    
    if match:
        year, month, day = map(int, match[0])

        # 날짜 객체로 변환
        date_object = datetime(2000 + year, month, day) 
        formatted_date = date_object.strftime('%Y-%m-%d')
    else:
        formatted_date = ''  # 또는 다른 기본값으로 설정
    
    return formatted_date, weekday, revisit


### 1. 그룹 데이터에서 검색어 사용( 정렬이 완료되었다는 가정 하에)

In [132]:
# res_group_df = pd.read_excel('매장_groupby_test.xlsx')
# res_group_df.drop('Unnamed: 1', axis=1, inplace=True)
# res_group_df


### NaN값, '전처리 필요' 없애기 ( vscode 컨디션 안좋아서 크롤링 안된경우 있음)



In [133]:
# res_group_df = res_group_df.loc[res_group_df['주소'] != '전처리필요'].dropna()
# res_df = res_group_df.copy()
# res_df = res_df[2:3] # test용
# res_df = res_df.reset_index(drop=True) # 인덱스 새로하기
# res_df

In [134]:
# 데이터 불러오기
original_res_df = pd.read_excel('combined_file.xlsx')
res_df = original_res_df.copy()
res_df = res_df.loc[res_df['검색여부'] == '가능'].reset_index(drop=True)
res_df.drop('Unnamed: 0', axis=1, inplace=True)


# 컬럼 순서 바꾸기 -> 보기 좋으라고 바꿈
res_df['검색어'] = res_df['상세주소'] + " " + res_df['사업장명']
column_order = ['검색어', '업태구분명', '사업장명', '사업장명_원본', '시도', '지역구', '상세주소', '소재지전체주소', '도로명전체주소', '좌표정보(x)', '좌표정보(y)', '검색여부', '영업상태명']
res_df_or = res_df[column_order]
res_df_or.head(3)

,검색어,업태구분명,사업장명,사업장명_원본,시도,지역구,상세주소,소재지전체주소,도로명전체주소,좌표정보(x),좌표정보(y),검색여부,영업상태명
0,"강남대로114길 18, 정식당",한식,정식당,정식당,서울특별시,강남구,"강남대로114길 18,",서울특별시 강남구 논현동 198-9,"서울특별시 강남구 강남대로114길 18, 지상1층 102호 (논현동)",202203.971845,444853.761458,가능,영업/정상
1,"강남대로 378, 채선당 샤브보트",한식,채선당 샤브보트,채선당 샤브보트 강남역점,서울특별시,강남구,"강남대로 378,",서울특별시 강남구 역삼동 825-9 준빌딩,"서울특별시 강남구 강남대로 378, 준빌딩 지상1층 (역삼동)",202462.794327,443827.961945,가능,영업/정상
2,"압구정로 343, 수아당",한식,수아당,수아당,서울특별시,강남구,"압구정로 343,",서울특별시 강남구 압구정동 494 갤러리아백화점,"서울특별시 강남구 압구정로 343, 갤러리아백화점 지하1층 (압구정동)",203470.848439,447369.579852,가능,영업/정상


In [135]:
res_df = res_df_or.copy()
res_df = res_df[0:2] # test용
res_df = res_df.reset_index(drop=True) # 인덱스 새로하기
res_df

,검색어,업태구분명,사업장명,사업장명_원본,시도,지역구,상세주소,소재지전체주소,도로명전체주소,좌표정보(x),좌표정보(y),검색여부,영업상태명
0,"강남대로114길 18, 정식당",한식,정식당,정식당,서울특별시,강남구,"강남대로114길 18,",서울특별시 강남구 논현동 198-9,"서울특별시 강남구 강남대로114길 18, 지상1층 102호 (논현동)",202203.971845,444853.761458,가능,영업/정상
1,"강남대로 378, 채선당 샤브보트",한식,채선당 샤브보트,채선당 샤브보트 강남역점,서울특별시,강남구,"강남대로 378,",서울특별시 강남구 역삼동 825-9 준빌딩,"서울특별시 강남구 강남대로 378, 준빌딩 지상1층 (역삼동)",202462.794327,443827.961945,가능,영업/정상


### 업태구분별로 3개씩 url 가져오기

In [136]:
# webdriver_manager를 사용하여 ChromeDriver 다운로드 및 설정
driver = webdriver.Chrome(ChromeDriverManager().install())

# # webdriver_manager를 사용하여 ChromeDriver 다운로드 및 설정
# import chromedriver_autoinstaller
# chromedriver_autoinstaller.install()
# driver = webdriver.Chrome()

user_url_info_df = pd.DataFrame()
search_name_list = []
user_url_list = []
user_name_list = []

for i in tqdm(range(0, len(res_df)), desc='진행 중', position=0, leave=True):
    name = res_df['검색어'][i]
    driver.get('https://map.naver.com/p/search/{}'.format(name))
    time.sleep(3)

    try:
        if driver.find_elements(By.ID, 'entryIframe'):
            entryIframe = driver.find_element(By.ID, 'entryIframe')
            driver.switch_to.frame(entryIframe)
            time.sleep(1)

        # url이 나타날때까지 스크롤 내리기 (컨디션을 좀 타서 안될때도 있음)
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(3)

        # html 긁어오기
        html_code = driver.page_source
        soup = BeautifulSoup(html_code, 'html.parser')

        # 리뷰들이 포함된 <li> 태그들을 선택
        reviews = soup.select('ul.Uf1BQ li.qrzj_')

        for review in reviews:
            # 리뷰 페이지의 href 가져오기
            user_url = review.select_one('div.ozEbl a.iz4rh').get('href')
            user_name = review.select_one('div.rg88i').text

            ## 리스트 ㄱ
            search_name_list.append(name)
            user_url_list.append(user_url)
            user_name_list.append(user_name)

    except Exception as e:
        print(f"에러 메시지: {str(e)}")


user_url_info_df = pd.DataFrame({'검색어' : search_name_list,
                                 '아이디' : user_name_list,
                             'url' : user_url_list
                             })

    
user_url_info_df


진행 중: 100%|██████████| 2/2 [00:16<00:00,  8.22s/it]


,검색어,아이디,url
0,"강남대로114길 18, 정식당",hsy****,https://m.place.naver.com/my/5efe11de854ca9293...
1,"강남대로114길 18, 정식당",arias7932,https://m.place.naver.com/my/5edf44e48f87a842b...
2,"강남대로114길 18, 정식당",퍼플리보,https://m.place.naver.com/my/5c5d452d3ea9594c7...
3,"강남대로 378, 채선당 샤브보트",don****,https://m.place.naver.com/my/652000e28314d60a9...
4,"강남대로 378, 채선당 샤브보트",체리는 맛있어,https://m.place.naver.com/my/5da0a1628f87a842b...
5,"강남대로 378, 채선당 샤브보트",하늘빛눈,https://m.place.naver.com/my/5d251d01dd1a688ac...


In [137]:
# 유저 리뷰 가져오는 함수 
def find_user_data(page_down,driver) : 
    # 맨 처음 게시물 클릭 
    button = driver.find_element(By.CLASS_NAME, '_3P-5HQ')
    button.click()
    time.sleep(3)
    html = driver.page_source
    soup = BeautifulSoup(driver.page_source, 'html.parser')

    # data를 담을 빈 리스트 선언
    data_list = [] 

    # 버튼을 클래스로 찾아 리스트로 가져오기
    buttons = driver.find_elements(By.CLASS_NAME, '_3P-5HQ')
    if buttons:
        first_button = buttons[0]
        first_button.click()

    # 페이지 아래로 내리기 
    for _ in range(page_down):

        # '내용더보기' 버튼들 찾고 죄다 클릭해두기  
        try :
            button_element = WebDriverWait(driver, 10).until(
                EC.presence_of_all_elements_located((By.CLASS_NAME, 'wTaI4v._1dVUyi'))
            )
            for button in button_element:
                button.click()
        except :
            pass 

        # 다음 페이지로 잘 넘어갔는지 기다리기 
        wait = WebDriverWait(driver, 10)
        user_review_elements = wait.until(
            EC.presence_of_all_elements_located((By.CLASS_NAME, '_27tH92')))
        driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.PAGE_DOWN) 


        # 리뷰에서 정보 가져오기
        for reviews_elements in user_review_elements:
            i = 0 # 카테고리와 주소 구분할 때 사용
            restaurant_elements=reviews_elements.find_elements(By.CLASS_NAME, '_1QGRWW')
        
            if bool(restaurant_elements): # 정보가 없는 경우가 있음

                # 매장명 찾기
                restaurant_name = restaurant_elements[0].text  

        #<--------------------------------------------------------------------------------------------------->
                # 카테고리와 주소 찾기 
                category_location_elements = reviews_elements.find_elements(By.CLASS_NAME, '_2vBfgu')
                i = 0  # i 초기화
                for category_location_element in category_location_elements:
                    category_location_soup = BeautifulSoup(category_location_element.get_attribute('outerHTML'), 'html.parser')
                
                    # span 태그 안에 있는 텍스트 가져오기
                    span_elements = category_location_soup.find_all('span', class_='wzFIfJ')
                    category = span_elements[0].text if span_elements and len(span_elements) > 0 else '없음'
                    location = span_elements[1].text if span_elements and len(span_elements) > 1 else '없음'
                #<--------------------------------------------------------------------------------------------------->

                    review_elements = reviews_elements.find_elements(By.CLASS_NAME,'z0t_8b')
                    try : 
                        review_text = review_elements[0].text # 리뷰 본문      
                        review_text = remove_special_characters(review_text)
                        sub_info_elements = reviews_elements.find_elements(By.CLASS_NAME,'_1tkuel') # 세부정보
                        sub_info = sub_info_elements[0].text if sub_info_elements else None
                    except :
                        review_text = '리뷰 없음'
                        sub_info = ''
                #<--------------------------------------------------------------------------------------------------->
                    # tag 찾기 
                    reactions_elements = reviews_elements.find_elements(By.CLASS_NAME, 'COw42b')  # 리엑션 리스트
                    command = False

                    for x in reactions_elements:
                        if x.get_attribute("role"):  # 리액션 숨김 없애기
                            try:
                                # 요소가 클릭 가능할 때까지 대기
                                wait = WebDriverWait(driver, 10)
                                reaction_element = wait.until(EC.element_to_be_clickable((By.CLASS_NAME, 'COw42b')))
                                
                                # JavaScript를 사용하여 요소 클릭
                                driver.execute_script("arguments[0].click();", reaction_element)
                                
                                command = True
                            except:
                                continue

                    if command:
                        # 리엑션 리스트 재탐색
                        reactions_elements = reviews_elements.find_elements(By.CLASS_NAME, 'COw42b')

                    # 리스트를 벗김
                    reactions = list(map(lambda x: x.text, reactions_elements))
                    reactions = str(reactions).replace('[', '').replace(']', '')                    

                #<--------------------------------------------------------------------------------------------------->
                    # 방문 일자, 재방문 이력 찾기
                    date_info = reviews_elements.find_element(By.CLASS_NAME, '_15xwjO .hol3Ic').find_elements(By.CLASS_NAME,'_3nNYBi')
                    date=[x.text for x in date_info]
                    day,weekday,revisit = change_date_format(date) # 함수 사용

                    global user_data

                    # 정보 추가
                    data_dict = {
                        '아이디' : user_data['아이디'],
                        '매장명': restaurant_name,
                        '카테고리': category,
                        '주소' : location,
                        '리뷰 내용': review_text,
                        "세부정보" : sub_info,
                        '태그': reactions,
                        '방문일자': day,
                        '요일' : weekday,
                        '재방문횟수':revisit
                    }
                    data_list.append(data_dict)

        user_df = pd.DataFrame(data_list)
        user_df = user_df.drop_duplicates()
  
    return user_df



### url에 들어가서 유저 정보와 리뷰 저장

In [139]:
# webdriver_manager를 사용하여 ChromeDriver 다운로드 및 설정
driver = webdriver.Chrome(ChromeDriverManager().install()) 

# driver = webdriver.Chrome()

# 스크롤 횟수 
page_down = 3

# 데이터프레임 선언
total_user_df =  pd.DataFrame()
user_profile_df = pd.DataFrame(columns=['아이디', '리뷰', '팔로워', '팔로잉', 'url'])

# 반복해서 url 정보 가져오기 
for url in tqdm(user_url_info_df['url'], desc='진행 중', position=0, leave=True):
    try: 
        # User의 아이디, 리뷰수, 팔로워를 가져오는 함수  
        user_data = user_profile(url)
        
        # 데이터프레임으로 만들기
        user_data_df = pd.DataFrame([user_data])
        user_profile_df = pd.concat([user_profile_df, user_data_df], ignore_index=True)
        
        # User review 정보를 가져오는 함수
        user_df = find_user_data(page_down, driver)

        # User review 정보 저장 
        total_user_df = pd.concat([total_user_df, user_df], ignore_index=True)

    except Exception as e:
        print(f"에러 메시지: {str(e)}")
        continue


진행 중:   0%|          | 0/6 [00:00<?, ?it/s]

{'아이디': 'hsy****', '리뷰': 11, '팔로워': 0, '팔로잉': 0, 'url': 'https://m.place.naver.com/my/5efe11de854ca929311318ac/review?v=2'}


진행 중:  17%|█▋        | 1/6 [00:28<02:21, 28.30s/it]

{'아이디': 'arias7932', '리뷰': 8, '팔로워': 0, '팔로잉': 0, 'url': 'https://m.place.naver.com/my/5edf44e48f87a842bc04f794/review?v=2'}


진행 중:  33%|███▎      | 2/6 [01:05<02:13, 33.49s/it]

...

진행 중:  67%|██████▋   | 4/6 [02:08<01:02, 31.42s/it]

{'아이디': '체리는 맛있어', '리뷰': 20, '팔로워': 2, '팔로잉': 0, 'url': 'https://m.place.naver.com/my/5da0a1628f87a842bcae4f8e/review?v=2'}


진행 중:  83%|████████▎ | 5/6 [02:38<00:30, 30.85s/it]

{'아이디': '하늘빛눈', '리뷰': 353, '팔로워': 0, '팔로잉': 2, 'url': 'https://m.place.naver.com/my/5d251d01dd1a688aca746736/review?v=2'}


진행 중: 100%|██████████| 6/6 [03:22<00:00, 33.79s/it]


In [140]:
user_profile_df

,아이디,리뷰,팔로워,팔로잉,url
0,hsy****,11,0,0,https://m.place.naver.com/my/5efe11de854ca9293...
1,arias7932,8,0,0,https://m.place.naver.com/my/5edf44e48f87a842b...
2,퍼플리보,2,0,0,https://m.place.naver.com/my/5c5d452d3ea9594c7...
3,don****,41,0,0,https://m.place.naver.com/my/652000e28314d60a9...
4,체리는 맛있어,20,2,0,https://m.place.naver.com/my/5da0a1628f87a842b...
5,하늘빛눈,353,0,2,https://m.place.naver.com/my/5d251d01dd1a688ac...


In [141]:
total_user_df

,아이디,매장명,카테고리,주소,리뷰 내용,세부정보,태그,방문일자,요일,재방문횟수
0,hsy****,정식당,한식,서울특별시 강남구 논현동,친구랑 어디갈까 고민하다 오게됬어요~ 근데 사진엔 없지만 불고기 진짜 미쳤구요.. ...,"이용 방법예약 없이대기 시간바로 입장목적친목, 기념일동행연인·배우자, 친구, 지인·동료","'음식이 맛있어요', '재료가 신선해요', '가성비가 좋아요', '친절해요', '오...",,목요일,0
1,hsy****,여봉닭발 중랑본점,닭발,서울특별시 중랑구 면목동,닭발귀신인데 닭발땡겨서 퇴근런하고 왔어요\n여긴 이지역에서 워낙유명해서 역시나 줄스...,"이용 방법예약 없이대기 시간30분 이내목적친목동행연인·배우자, 친구",'음식이 맛있어요',,목요일,0
2,hsy****,이여사네 김치/반찬,반찬가게,서울특별시 강남구 일원동,친구가 맛있다고 추천해줘서 남편이랑 같이 방문했는데 우선 너무 청결해서 좋았구요~ ...,None,"'재료가 신선해요', '위생적이에요', '음식이 맛있어요', '가성비가 좋아요', ...",2022-07-11,월요일,0
3,arias7932,정식당,한식,서울특별시 강남구 논현동,신논현 찐 맛집!!~오픈한지 얼마 안되었다기에 여자3명이 방문~요즘 물가가 많이 올...,이용 방법예약 없이대기 시간바로 입장목적친목동행친구,"'음식이 맛있어요', '재료가 신선해요', '가성비가 좋아요', '친절해요', '화...",,월요일,0
4,arias7932,고기이야기본점※강남곱창이야기,없음,없음,직원분들이 친절하고 넘 좋았어요^^가성비 갑\n고기두 맛나고~알고보니 곱창이야기 사...,,,,5,19327
...,...,...,...,...,...,...,...,...,...,...
63,하늘빛눈,만랩커피 용산문배KCC점,카페,서울특별시 용산구 문배동,양도많고 맛나옹^^*,이용 방법예약 없이목적일상동행혼자,"'커피가 맛있어요', '+1\n개의 리뷰 더 있습니다'",,일요일,0
64,하늘빛눈,크리스피크림도넛 남영역점,도넛,서울특별시 용산구 갈월동,리뷰 없음,,"'커피가 맛있어요', '디저트가 맛있어요', '음료가 맛있어요', '인테리어가 멋져요'",,일요일,0
65,하늘빛눈,커피빈 뉴압구정점,카페,서울특별시 강남구 신사동,역시 커피빈은 바닐라라떼!ㅎ 취향에 맞게 초코가루를 듬뿍 ㅎㅎ 달달초과,이용 방법예약 없이대기 시간바로 입장목적일상동행혼자,"'인테리어가 멋져요', '+3\n개의 리뷰 더 있습니다'",,토요일,0
66,하늘빛눈,파머스마켓팥집,한식,서울특별시 강남구 신사동,속초비빔국수! 먹어보세용^^ 맛나용,이용 방법예약 없이대기 시간바로 입장목적일상동행혼자,"'음식이 맛있어요', '+2\n개의 리뷰 더 있습니다'",,토요일,0


In [144]:
total_user_df.iloc[4]

아이디                                              arias7932
매장명                                        고기이야기본점※강남곱창이야기
카테고리                                                    없음
주소                                                      없음
리뷰 내용    직원분들이 친절하고 넘 좋았어요^^가성비 갑\n고기두 맛나고~알고보니 곱창이야기 사...
세부정보                                                      
태그                                                        
방문일자                                                      
요일                                                       5
재방문횟수                                                19327
Name: 4, dtype: object

In [142]:
# 유저의 리뷰
total_user_df.to_excel('data/user_review_data/user_df_test.xlsx', index=False)

# 유저의 정보 
user_profile_df.to_excel('data/user_review_data/user_profile_df_test.xlsx', index=False)